<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    Auxiliar script for releases, revisions and more
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h3>
        Documentation
        <br>
        ____________________
            </br>
    </h3>
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> This auxiliary script serves to load the main project’s input data into SQL, and to merge (concatenate) both the old and new GDP datasets, along with the release datasets. However, the most significant advancement lies in the creation of cumulative and ongoing revisions data, which is derived from the outputs of the main project scripts. Additionally, this script should be viewed as a non-trivial extension to the remaining project scripts.<div/>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #94FFD8; font-size: 18px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #94FFD8; font-size: 18px;">
        Initial set-up</a>
    <br>
    <a href="#selector" style="color: #94FFD8; font-size: 18px;">
        Select <code>sector</code> and <code>frequency</code></a>
    <br>
    <a href="#1" style="color: #94FFD8; font-size: 18px;">
        1. Loading gdp_percentage_structure</a>
    <br>
    <a href="#2" style="color: #94FFD8; font-size: 18px;">
        2. Generate PDF input with key tables</a>
    <br>
    <a href="#3" style="color: #94FFD8; font-size: 18px;">    
        3. Load date, year, id and base year data</a>
    <br>
    <a href="#4" style="color: #94FFD8; font-size: 18px;">    
        4. Load dummies for raw data delivered by Central Bank</a>
    <br>
    <a href="#5" style="color: #94FFD8; font-size: 18px;">    
        5. Concatenate old_gdp_dataset with new_gdp_dataset</a>
    <br>
    <a href="#6" style="color: #94FFD8; font-size: 18px;">    
        6. Merge all the releases datasets</a>
    <br>
    <a href="#7" style="color: #94FFD8; font-size: 18px;">    
        7. Create cumulative revisions</a>
    <br>
    <a href="#8" style="color: #94FFD8; font-size: 18px;">    
        8. Create intermediate revisions</a>
</div>


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123)">email Jason Cruz
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px"">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import re

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following code lines will create folders in your current path, call them to import and export your outputs. <div/>

In [2]:
# Folder path to save the csv file of GDP percentage structure

gdp_perc_structure_folder = 'gdp_perc_structure'
if not os.path.exists(gdp_perc_structure_folder):
    os.makedirs(gdp_perc_structure_folder) # to create the folder (if it doesn't exist)

In [3]:
# Folder path to save the csv file of NS dates

ns_dates_folder = 'ns_dates'
if not os.path.exists(ns_dates_folder):
    os.makedirs(ns_dates_folder) # to create the folder (if it doesn't exist)

In [4]:
# Folder path to save the csv file of NS base year

ns_base_year_folder = 'ns_base_year'
if not os.path.exists(ns_base_year_folder):
    os.makedirs(ns_base_year_folder) # to create the folder (if it doesn't exist)

In [5]:
# Folder path to save the xlsx files delivered by Central Bank

data_delivered_folder = 'old_raw_data_delivered'
if not os.path.exists(data_delivered_folder):
    os.makedirs(data_delivered_folder) # to create the folder (if it doesn't exist)

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [6]:
# Get environment variables
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Check if all environment variables are defined
if not all([host, user, password]):
    raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Create connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>aux_files_to_sql_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok. Altough some functions are included in <code>gdp_revisions_datasets_functions.py</code>.<div/>

In [7]:
from aux_files_to_sql_functions import *
from gdp_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="selector">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Select <code>sector</code> and <code>frequency</code>
    </h2>
    <div/>

<div id="step-5-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Select economic sector</span>
  </div>

In [8]:
# Call the function to show the popup window
sector = show_option_window()
print("Selected economic sector:", sector)

Selected economic sector: gdp


<div id="step-5-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Select frequency</span>
  </div>

In [9]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

Selected frequency: monthly


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-5-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-5-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 5</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#5" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#5" style="color: #3772ff; text-decoration: none;">Go to <b>Section 5</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-6-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-6-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 6</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#6" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#6" style="color: #3772ff; text-decoration: none;">Go to <b>Section 6</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-7-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-7-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 7</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#7" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#7" style="color: #3772ff; text-decoration: none;">Go to <b>Section 7</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-8-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-8-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 8</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#8" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#8" style="color: #3772ff; text-decoration: none;">Go to <b>Section 8</b>.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading gdp_percentage_structure</span></h1>
    </div>

In [ ]:
# Loading csv data
gdp_percentage_structure = f'{gdp_perc_structure_folder}/gdp_percentage_structure.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
gdp_percentage_structure = pd.read_csv(gdp_percentage_structure, delimiter=';')
gdp_percentage_structure.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
gdp_percentage_structure.to_sql('gdp_percentage_structure', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load date, year and id data</span></h1>
    </div>

In [ ]:
# Loading csv data
csv_file = f'{ns_dates_folder}/dates.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
dates = pd.read_csv(csv_file, delimiter=';')
dates.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Convert the 'date' column to datetime type without time
dates['date'] = pd.to_datetime(dates['date'], format='%d/%m/%Y', errors='coerce')

# Check the data type of the 'date' column after conversion
print(dates['date'].dtype)

# Check if there are NaT values in the 'date' column
print(dates['date'].isnull().sum())

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
dates.to_sql('dates_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load date, year, id and base year data</span></h1>
    </div>

In [ ]:
# Loading csv data
csv_file = f'{ns_base_year_folder}/base_year.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
base_year = pd.read_csv(csv_file, delimiter=';')
base_year.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Convert the 'date' column to datetime type without time
base_year['date'] = pd.to_datetime(base_year['date'], format='%d/%m/%Y', errors='coerce')

# Check the data type of the 'date' column after conversion
print(base_year['date'].dtype)

# Check if there are NaT values in the 'date' column
print(base_year['date'].isnull().sum())

In [ ]:
# Change to str
base_year['year'] = base_year['year'].astype(str)
base_year['id_ns'] = base_year['id_ns'].apply(lambda x: f'{x:02}')
base_year['base_year'] = base_year['base_year'].astype(str)
base_year.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
base_year.to_sql('ns_base_year', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load dummies for raw data delivered by Central Bank</span></h1>
    </div>

In [ ]:
# Loading csv data
delivered_data_dummies = f'{data_delivered_folder}/old_raw_data_delivered.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
delivered_data_dummies = pd.read_csv(delivered_data_dummies, delimiter=';')
delivered_data_dummies.head(5)

In [ ]:
delivered_data_dummies.to_sql('old_raw_data_delivered', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">5.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Concatenate old_gdp_dataset with new_gdp_dataset</span></h1>
    </div>

<div id="steps-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-5-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-5-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Concatenate dataframes</a>
    <br>
    <a href="#step-5-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Load to PostgresSQL</a>
</div>

<div id="step-5-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

In [ ]:
# Crear un DataFrame de pandas desde la tabla
query_1 = f"SELECT * FROM old_{sector}_{frequency}_growth_rates;"
df_1 = pd.read_sql(query_1, engine)

In [ ]:
# Crear un DataFrame de pandas desde la tabla
query_2 = f"SELECT * FROM new_{sector}_{frequency}_growth_rates;"
df_2 = pd.read_sql(query_2, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-5-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Concatenate dataframes</span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Run the function to create concatenated dataframe
    <div/>

In [ ]:
def concatenate_dataframes(df1, df2):
    # Concatenate the dataframes
    concatenated_df = pd.concat([df1, df2], ignore_index=True)
    
    return concatenated_df

# Call the function
globals()[f'{sector}_{frequency}_growth_rates'] = concatenate_dataframes(df_1, df_2)
df = globals()[f'{sector}_{frequency}_growth_rates']

# Display the resulting dataframe
df.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-5-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Load to PostgresSQL</span>
  </div>

In [ ]:
df.to_sql(f'{sector}_{frequency}_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
df.columns

In [ ]:
print(df['date'].dtype)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">6.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Merge all the releases datasets</span></h1>
    </div>

<div id="steps-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-6-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-6-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Merge dataframes</a>
    <br>
    <a href="#step-6-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Order by vintages_date</a>
    <br>
    <a href="#step-6-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Load to PostgresSQL</a>
</div>

<div id="step-6-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_1 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_1 = pd.read_sql(query_1, engine)
df_1.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_2 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_2 = pd.read_sql(query_2, engine)
df_2.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_3 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_3 = pd.read_sql(query_3, engine)
df_3.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_4 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_4 = pd.read_sql(query_4, engine)
df_4.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_5 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_5 = pd.read_sql(query_5, engine)
df_5.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_6 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_6 = pd.read_sql(query_6, engine)
df_6.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_7 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_7 = pd.read_sql(query_7, engine)
df_7.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_8 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_8 = pd.read_sql(query_8, engine)
df_8.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_9 = f"SELECT * FROM {sector}_{frequency}_releases;"
df_9 = pd.read_sql(query_9, engine)
df_9.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-6" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-6-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Merge dataframes</span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Run the function to create merged dataframe for releases
    <div/>

In [ ]:
def merge_dataframes_by_vintage(frequency, *dataframes):
    """
    Merges multiple dataframes on the 'vintages_date' column.

    Parameters:
    frequency (str): The frequency to be used in the resulting dataframe name.
    *dataframes (pd.DataFrame): DataFrames to be merged.

    Returns:
    pd.DataFrame: The merged dataframe with the name 'sectorial_gdp_{frequency}_releases'.
    """
    # Initialize the merged dataframe with the first dataframe
    merged_df = dataframes[0]

    # Merge each dataframe on 'vintages_date'
    for df in dataframes[1:]:
        merged_df = pd.merge(merged_df, df, on='vintages_date', how='outer')

    # Define the name of the resulting dataframe
    result_name = f'sectorial_gdp_{frequency}_releases'
    
    # Assign the name to the dataframe (this is for reference, actual DataFrame doesn't have a 'name' attribute)
    merged_df.name = result_name

    return merged_df

In [ ]:
merged_df = merge_dataframes_by_vintage('monthly', df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9)
merged_df.head(20)

In [ ]:
merged_df.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-6" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-6-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Order by vintages_date</span>
  </div>

In [ ]:
def sort_by_vintages_date(df):
    """
    Sorts the DataFrame by the 'vintages_date' column in ascending order.

    Parameters:
    df (pd.DataFrame): Input DataFrame with a datetime64[ns] column named 'vintages_date'.

    Returns:
    pd.DataFrame: Sorted DataFrame.
    """
    if 'vintages_date' not in df.columns:
        raise ValueError("The DataFrame does not contain a 'vintages_date' column.")
    
    if not pd.api.types.is_datetime64_ns_dtype(df['vintages_date']):
        raise TypeError("'vintages_date' column is not of type datetime64[ns].")

    return df.sort_values(by='vintages_date').reset_index(drop=True)

In [ ]:
merged_df = sort_by_vintages_date(merged_df)
merged_df.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-6" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-6-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> <span>Load to PostgresSQL</span>
  </div>

In [ ]:
merged_df.to_sql(f'sectorial_gdp_{frequency}_releases', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-6" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-5-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="7">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">7.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create cumulative revisions</span></h1>
    </div>

<div id="steps-7">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-7-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-7-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Compute cumulative revisions</a>
    <br>
    <a href="#step-7-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Keep cumulative revisions</a>
    <br>
    <a href="#step-7-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Transpose dataframe in order to have nowcast error (cumulative revisions) of fixed events as columns</a>
</div>

<div id="step-7-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_releases_1 = f"SELECT * FROM sectorial_{sector}_{frequency}_releases;"
df_releases_1 = pd.read_sql(query_releases_1, engine)
df_releases_1.head(10) # showing the first 10 obs

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Filter releases dataset
    <div/>

In [ ]:
# Set dataframe inn date range
df_releases_filtered_1 = df_releases_1[(df_releases_1['vintages_date'] >= '1993-01-01') & (df_releases_1['vintages_date'] <= '2023-12-31')]

# Order the dataframe by the column 'vintages_date'
df_releases_filtered_1 = df_releases_filtered_1.sort_values(by='vintages_date')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-7-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Compute cumulative revisions</span>
  </div>

In [ ]:
# Apply the function to your dataframe and get max_release
df_rev_cum, max_release = calculate_cumulative_revisions(df_releases_filtered_1)

# Output the max_release for verification
print(f"The maximum release number found: {max_release}")

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    Computing specific cumulative revisions
    <div/>

In [ ]:
#df_rev_cum = calculate_specific_cumulative_revisions(df_rev_cum, frequency)
#df_rev_cum

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Inspect specific columns
    <div/>

In [ ]:
df_rev_cum.columns

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-7-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Keep cumulative revisions</span>
  </div>

In [ ]:
df_rev_cum = destring_date_column(df_rev_cum, frequency)  # You can change 'quarterly' to 'annual' or 'monthly'

In [ ]:
df_rev_cum.head(10)

<div id="7">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Keeping cumulative revisions and date column only 
    <div/>

In [ ]:
df_rev_cum = keep_revisions(df_rev_cum, frequency) # this is optional in order to compress dataframe columns

In [ ]:
df_rev_cum.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_rev_cum.to_sql(f'sectorial_gdp_{frequency}_cum_revisions', engine, index=False, if_exists='replace')

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To data panel
    <div/>

In [ ]:
df_rev_cum_panel = convert_to_panel(df_rev_cum)

In [ ]:
df_rev_cum_panel[['vintages_date', 'revision', 'gdp']].head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_rev_cum_panel.to_sql(f'sectorial_gdp_{frequency}_cum_revisions_panel', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-7-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> <span>Transpose dataframe in order to have nowcast error (cumulative revisions) of fixed events as columns</span>
  </div>

In [ ]:
df_fixed_events_cum = transpose_df(df_rev_cum, frequency)
df_fixed_events_cum.head(5)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Compute time trend column
    <div/>

In [ ]:
# Appling above function
df_fixed_events_cum = add_time_trend(df_fixed_events_cum)
df_fixed_events_cum.head(5)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Replace "-" by "_" in columns
    <div/>

In [ ]:
df_fixed_events_cum = replace_hyphen_in_columns(df_fixed_events_cum)
df_fixed_events_cum.head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Checking columns
    <div/>

In [ ]:
df_fixed_events_cum.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Remove NaN or zero columns
    <div/>

In [ ]:
df_fixed_events_cum_filtered = remove_nan_or_zero_float_columns(df_fixed_events_cum)
df_fixed_events_cum_filtered

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
# Verificar si la frecuencia es mensual
if frequency == 'monthly':
    n = 1000  # Ajustar el tamaño del chunk
    
    # Asegurar que 'time_trend' esté en cada chunk
    time_trend = df_fixed_events_cum_filtered[['time_trend']]  # Extraer la columna 'time_trend'
    
    for i, chunk in enumerate(range(0, df_fixed_events_cum_filtered.shape[1], n)):
        # Seleccionar el chunk de columnas
        chunk_df = df_fixed_events_cum_filtered.iloc[:, chunk:chunk+n]
        
        # Verificar si 'time_trend' ya está en el chunk, si no, agregarla
        if 'time_trend' not in chunk_df.columns:
            chunk_df = pd.concat([time_trend, chunk_df], axis=1)
        
        # Guardar en SQL
        chunk_df.to_sql(f'cum_{frequency}_fixed_event_part_{i}', engine, index=False, if_exists='replace')

# Verificar si la frecuencia es anual o trimestral
elif frequency in ['annual', 'quarterly']:
    # Guardar el DataFrame completo en SQL
    df_fixed_events_cum_filtered.to_sql(f'cum_{frequency}_fixed_event', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="8">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">8.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create intermediate (ongoing) revisions</span></h1>
    </div>

<div id="steps-8">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-8-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-8-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Compute intermediate revisions</a>
    <br>
    <a href="#step-8-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Keep intermediate revisions</a>
    <br>
    <a href="#step-8-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Transpose dataframe in order to have nowcast error (intermediate revisions) of fixed events as columns</a>
</div>

<div id="step-8-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

In [10]:
# Create a pandas DataFrame from the SQL table
query_releases_2 = f"SELECT * FROM sectorial_{sector}_{frequency}_releases;"
df_releases_2 = pd.read_sql(query_releases_2, engine)
df_releases_2.head(10) # showing the first 10 obs

,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,...,services_release_10,services_release_11,services_release_12,services_release_13,services_release_14,services_release_15,services_release_16,services_release_17,services_release_18,services_most_recent
0,1992-01-01,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-02-01,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-03-01,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-04-01,-7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-05-01,-5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1992-06-01,-3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1992-07-01,-7.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1992-08-01,-5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1992-09-01,-4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1992-10-01,-3.6,-3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Filter releases dataset
    <div/>

In [11]:
# Set dataframe inn date range
df_releases_filtered_2 = df_releases_2[(df_releases_2['vintages_date'] >= '1993-01-01') & (df_releases_2['vintages_date'] <= '2023-12-31')]

# Order the dataframe by the column 'vintages_date'
df_releases_filtered_2 = df_releases_filtered_2.sort_values(by='vintages_date')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-8-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Compute intermediate revisions</span>
  </div>

In [12]:
df_aux, max_release = calculate_intermediate_revisions(df_releases_filtered_2)
# Output the max_release for verification
print(f"The maximum release number found: {max_release}")

The maximum release number found: 19


In [13]:
# Apply the function to your dataframe and get max_release
df_rev_int, max_release = calculate_intermediate_revisions(df_releases_filtered_2)

# Output the max_release for verification
print(f"The maximum release number found: {max_release}")

The maximum release number found: 19


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Computing specific intermediate revisions
    <div/>

In [14]:
#df_rev_int = calculate_specific_intermediate_revisions(df_rev_int, frequency)
#df_rev_int

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Inspect specific columns
    <div/>

In [15]:
df_rev_int[['vintages_date', 'agriculture_release_1', 'agriculture_release_2', 'r_2_agriculture']]

,vintages_date,agriculture_release_1,agriculture_release_2,r_2_agriculture
12,1993-01-01,-1.7,-1.7,0.0
13,1993-02-01,0.8,0.8,0.0
14,1993-03-01,-4.1,-4.1,0.0
15,1993-04-01,-2.4,-2.4,0.0
16,1993-05-01,0.9,0.9,0.0
...,...,...,...,...
379,2023-08-01,-4.0,-6.4,-2.4
380,2023-09-01,-8.8,-8.8,0.0
381,2023-10-01,-7.6,-7.6,0.0
382,2023-11-01,1.2,2.7,1.5


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-8-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Keep intermediate revisions</span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Use <code>destring_date_column</code> only if you are going to transpose the dataframe later
    <div/>

In [16]:
df_rev_int = destring_date_column(df_rev_int, frequency)  # You can change 'quarterly' to 'annual' or 'monthly'

In [17]:
df_rev_int.head(10)

,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,...,r_12_agriculture,r_13_agriculture,r_14_agriculture,r_15_agriculture,r_16_agriculture,r_17_agriculture,r_18_agriculture,r_19_agriculture,r_20_agriculture,year_month
12,1993-01-01,1.2,1.3,1.3,1.3,-2.2,-2.2,-2.2,-2.2,-2.2,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-01
13,1993-02-01,0.1,0.2,0.1,0.2,-1.6,-1.6,-1.6,-1.6,-1.6,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-02
14,1993-03-01,5.7,5.8,5.9,5.9,5.0,5.0,5.0,5.0,4.9,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-03
15,1993-04-01,7.2,7.2,7.3,7.3,6.0,6.0,6.0,6.0,6.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-04
16,1993-05-01,4.5,4.8,4.7,4.7,3.9,3.9,3.9,3.9,4.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-05
17,1993-06-01,7.6,8.2,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-06
19,1993-07-01,11.6,12.0,11.9,11.9,11.5,11.6,11.6,11.5,11.3,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-07
20,1993-08-01,11.2,11.8,11.8,11.8,12.5,12.5,12.5,12.5,12.6,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-08
21,1993-09-01,10.5,10.4,10.6,10.6,10.2,10.2,10.2,10.2,9.9,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-09
22,1993-10-01,6.3,5.4,5.4,5.4,4.9,4.8,4.8,4.9,4.8,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1993-10


<div id="7">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Keeping intermediate revisions and date column only 
    <div/>

In [18]:
df_rev_int = keep_revisions(df_rev_int, frequency) # this is optional in order to compress dataframe columns

In [19]:
df_rev_int.columns

Index(['vintages_date', 'year_month', 'r_2_electricity', 'r_3_electricity',
       'r_4_electricity', 'r_5_electricity', 'r_6_electricity',
       'r_7_electricity', 'r_8_electricity', 'r_9_electricity',
       ...
       'r_11_agriculture', 'r_12_agriculture', 'r_13_agriculture',
       'r_14_agriculture', 'r_15_agriculture', 'r_16_agriculture',
       'r_17_agriculture', 'r_18_agriculture', 'r_19_agriculture',
       'r_20_agriculture'],
      dtype='object', length=165)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_rev_int.to_sql(f'sectorial_gdp_{frequency}_int_revisions', engine, index=False, if_exists='replace')

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To data panel
    <div/>

In [20]:
df_rev_int_panel = int_convert_to_panel(df_rev_int)
df_rev_int_panel

,vintages_date,revision,mining,electricity,gdp,services,agriculture,commerce,construction,fishing,manufacturing
0,1993-01-01,2,1.0,NaN,0.1,NaN,0.0,0.0,0.0,0.0,0.0
1,1993-02-01,2,0.6,NaN,0.1,NaN,0.0,0.1,0.0,0.0,0.0
2,1993-03-01,2,0.5,NaN,0.1,NaN,0.0,0.0,0.0,0.0,0.0
3,1993-04-01,2,0.2,NaN,0.0,NaN,0.0,-0.1,0.0,0.0,0.0
4,1993-05-01,2,2.0,NaN,0.3,NaN,0.0,0.1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7063,2023-08-01,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7064,2023-09-01,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7065,2023-10-01,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7066,2023-11-01,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_rev_int_panel['vintages_date'].dtype

dtype('<M8[ns]')

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [24]:
df_rev_int_panel.to_sql(f'sectorial_gdp_{frequency}_int_revisions_panel', engine, index=False, if_exists='replace')

68

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-7-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> <span>Transpose dataframe in order to have nowcast error (intermediate revisions) of fixed events as columns</span>
  </div>

In [ ]:
df_fixed_events_int = transpose_df(df_rev_int, frequency)
df_fixed_events_int.head(5)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Compute time trend column
    <div/>

In [ ]:
# Appling above function
df_fixed_events_int = add_time_trend(df_fixed_events_int)
df_fixed_events_int.head(5)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Replace "-" by "_" in columns
    <div/>

In [ ]:
df_fixed_events_int = replace_hyphen_in_columns(df_fixed_events_int)
df_fixed_events_int.head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Checking columns
    <div/>

In [ ]:
df_fixed_events_int.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Remove NaN or zero columns
    <div/>

In [ ]:
df_fixed_events_int_filtered = remove_nan_or_zero_float_columns(df_fixed_events_int)
df_fixed_events_int_filtered

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
# Verificar si la frecuencia es mensual
if frequency == 'monthly':
    n = 1000  # Ajustar el tamaño del chunk
    
    # Asegurar que 'time_trend' esté en cada chunk
    time_trend = df_fixed_events_int_filtered[['time_trend']]  # Extraer la columna 'time_trend'
    
    for i, chunk in enumerate(range(0, df_fixed_events_int_filtered.shape[1], n)):
        # Seleccionar el chunk de columnas
        chunk_df = df_fixed_events_int_filtered.iloc[:, chunk:chunk+n]
        
        # Verificar si 'time_trend' ya está en el chunk, si no, agregarla
        if 'time_trend' not in chunk_df.columns:
            chunk_df = pd.concat([time_trend, chunk_df], axis=1)
        
        # Guardar en SQL
        chunk_df.to_sql(f'int_{frequency}_fixed_event_part_{i}', engine, index=False, if_exists='replace')

# Verificar si la frecuencia es anual o trimestral
elif frequency in ['annual', 'quarterly']:
    # Guardar el DataFrame completo en SQL
    df_fixed_events_int_filtered.to_sql(f'int_{frequency}_fixed_event', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

# Regression

In [ ]:
pd.DataFrame(df_fixed_events_cum.filter(like='gdp_'))

In [ ]:
# Crear la variable dependiente (el cuadrado de 'services_2021')
df_fixed_events_cum['services_2021_abs'] = df_fixed_events_cum['services_2021'].abs()
df_fixed_events_cum

In [ ]:
import statsmodels.api as sm

# Eliminar filas con NaN en las columnas necesarias para la regresión
clean_df = fixed_events_df[['services_2021_abs', 'time_trend']].dropna()

# Definir la variable independiente ('time_trend')
X = clean_df['time_trend']

# Agregar una constante para el término alpha (intercepto)
X = sm.add_constant(X)

# Definir la variable dependiente
y = clean_df['services_2021_abs']

# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Imprimir los resultados del modelo
print(model.summary())

In [ ]:
df_filtered = df_fixed_events_1.iloc[:20]
df_filtered

In [ ]:
import statsmodels.api as sm

# Eliminar filas con NaN en las columnas necesarias para la regresión
clean_df = df_filtered[['services_2021_abs', 'time_trend']].dropna()

# Definir la variable independiente ('time_trend')
X = clean_df['time_trend']

# Agregar una constante para el término alpha (intercepto)
X = sm.add_constant(X)

# Definir la variable dependiente
y = clean_df['services_2021_abs']

# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Imprimir los resultados del modelo
print(model.summary())

In [ ]:
df_removed_columns = remove_nan_or_zero_float_columns(df_filtered_1)
df_removed_columns

In [ ]:
pd.DataFrame(df_removed_columns.filter(like='gdp_'))

# Newey West

In [ ]:
# Crear la variable dependiente (el cuadrado de 'services_2021')
df_fixed_events_cum_filtered['gdp_2002_abs'] = df_fixed_events_cum['gdp_2002'].abs()
df_fixed_events_cum_filtered

In [ ]:
import statsmodels.api as sm

In [ ]:
# Eliminar filas con NaN en las columnas necesarias para la regresión
clean_df = df_fixed_events_cum_filtered[['gdp_2002_abs', 'time_trend']].dropna()

# Suponiendo que clean_df es tu DataFrame ya limpio
X = clean_df['time_trend']
y = clean_df['gdp_2002_abs']

# Agregar una constante a la variable independiente
X = sm.add_constant(X)

# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Calcular errores estándar de Newey-West
newey_west_cov = model.get_robustcov_results(cov_type='HAC', maxlags=1)

# Imprimir los resultados con errores estándar de Newey-West
print(newey_west_cov.summary())